In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('A.csv')
del df['DATETIME']
del df['F_PU9']
del df['S_PU9']
del df['F_PU5']
del df['S_PU5']
del df['F_PU3']
del df['S_PU3']
del df['S_PU1']
df.head()

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,F_PU2,S_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,2.44,5.24,3.19,4.10,2.86,5.50,4.39,93.63,93.65,1,...,70.00,28.22,85.87,21.69,82.72,21.58,71.99,39.33,29.64,0
1,2.66,4.53,3.20,4.18,3.29,5.44,4.53,89.41,89.43,1,...,87.73,24.45,84.87,29.81,86.62,29.81,59.76,42.17,26.15,0
2,3.11,3.66,3.66,4.21,3.87,5.15,3.22,89.88,89.89,1,...,89.29,23.90,87.11,29.85,87.64,29.85,58.50,42.00,25.56,0
3,3.62,3.04,4.17,4.04,3.56,4.98,2.40,88.10,88.12,1,...,91.98,27.10,68.75,31.60,64.25,31.47,72.30,43.24,28.38,0
4,4.08,2.68,4.73,3.20,3.11,5.39,3.46,87.01,87.03,1,...,92.11,26.76,68.74,32.30,64.23,32.17,72.53,44.00,28.04,0


In [3]:
df=df.dropna()

In [4]:
df['ATT_FLAG'].value_counts()

ATT_FLAG
0    3685
1     492
Name: count, dtype: int64

In [5]:
df.columns

Index(['L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7', 'F_PU1',
       'F_PU2', 'S_PU2', 'F_PU4', 'S_PU4', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7',
       'F_PU8', 'S_PU8', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2',
       'S_V2', 'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415',
       'P_J302', 'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422', 'ATT_FLAG'],
      dtype='object')

In [6]:
df.head()

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,F_PU2,S_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,2.44,5.24,3.19,4.10,2.86,5.50,4.39,93.63,93.65,1,...,70.00,28.22,85.87,21.69,82.72,21.58,71.99,39.33,29.64,0
1,2.66,4.53,3.20,4.18,3.29,5.44,4.53,89.41,89.43,1,...,87.73,24.45,84.87,29.81,86.62,29.81,59.76,42.17,26.15,0
2,3.11,3.66,3.66,4.21,3.87,5.15,3.22,89.88,89.89,1,...,89.29,23.90,87.11,29.85,87.64,29.85,58.50,42.00,25.56,0
3,3.62,3.04,4.17,4.04,3.56,4.98,2.40,88.10,88.12,1,...,91.98,27.10,68.75,31.60,64.25,31.47,72.30,43.24,28.38,0
4,4.08,2.68,4.73,3.20,3.11,5.39,3.46,87.01,87.03,1,...,92.11,26.76,68.74,32.30,64.23,32.17,72.53,44.00,28.04,0


In [7]:
x1 = df.drop(labels='ATT_FLAG', axis=1).values
y1 = df.loc[:,'ATT_FLAG'].values

In [8]:
# Data Preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features = scaler.fit_transform(x1)

In [9]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros =RandomOverSampler(random_state=42)
x,y=ros.fit_resample(x1,y1)
print("OUR DATASET COUNT         : ", Counter(y1))
print("OVER SAMPLING DATA COUNT  : ", Counter(y))

OUR DATASET COUNT         :  Counter({0: 3685, 1: 492})
OVER SAMPLING DATA COUNT  :  Counter({0: 3685, 1: 3685})


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42, stratify=y)
print("NUMBER OF TRAIN DATASET    : ", len(x_train))
print("NUMBER OF TEST DATASET      : ", len(x_test))
print("TOTAL NUMBER OF DATASET    : ", len(x_train)+len(x_test))

NUMBER OF TRAIN DATASET    :  5896
NUMBER OF TEST DATASET      :  1474
TOTAL NUMBER OF DATASET    :  7370


In [11]:
print("NUMBER OF TRAIN DATASET    : ", len(y_train))
print("NUMBER OF TEST DATASET      : ", len(y_test))
print("TOTAL NUMBER OF DATASET    : ", len(y_train)+len(y_test))

NUMBER OF TRAIN DATASET    :  5896
NUMBER OF TEST DATASET      :  1474
TOTAL NUMBER OF DATASET    :  7370


In [12]:
# Convert features and labels to torch tensors
features = torch.tensor(x, dtype=torch.float32)
labels = torch.tensor(y, dtype=torch.long)


In [13]:
# Step 2: Split the dataset into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42
)


In [14]:
# Step 3: Create DataLoader for batch processing
batch_size = 32
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [15]:

# Step 4: Define the neural network model
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [16]:
# Step 5: Create the model and set hyperparameters
input_size = 36
hidden_size = 128
num_classes = 3
learning_rate = 0.001
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MyModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Initialize lists to store loss and accuracy for each epoch
epoch_losses = []
epoch_accuracies = []

In [17]:
from sklearn.metrics import accuracy_score

# Step 6: Training the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    num_batches = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num_batches += 1

    average_loss = running_loss / num_batches
    
    # Calculate accuracy at the end of each epoch
    model.eval()  # Switch to evaluation mode
    with torch.no_grad():
        test_predictions = []
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_predictions.extend(predicted.cpu().numpy())

        # Calculate accuracy
        accuracy = accuracy_score(test_labels, test_predictions)
        epoch_accuracies.append(accuracy)
        epoch_losses.append(average_loss)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy * 100:.2f}%, Loss: {average_loss:.4f}')

    model.train()  # Switch back to training mode

print('Training finished.')

Epoch [1/20], Accuracy: 62.62%, Loss: 1.0968
Epoch [2/20], Accuracy: 68.86%, Loss: 0.6228
Epoch [3/20], Accuracy: 62.35%, Loss: 0.6120
Epoch [4/20], Accuracy: 62.75%, Loss: 0.5837
Epoch [5/20], Accuracy: 69.81%, Loss: 0.5800
Epoch [6/20], Accuracy: 77.82%, Loss: 0.5491
Epoch [7/20], Accuracy: 70.56%, Loss: 0.5574
Epoch [8/20], Accuracy: 78.15%, Loss: 0.5516
Epoch [9/20], Accuracy: 67.64%, Loss: 0.5333
Epoch [10/20], Accuracy: 73.81%, Loss: 0.5219
Epoch [11/20], Accuracy: 76.26%, Loss: 0.5264
Epoch [12/20], Accuracy: 70.62%, Loss: 0.5350
Epoch [13/20], Accuracy: 68.86%, Loss: 0.5045
Epoch [14/20], Accuracy: 75.31%, Loss: 0.5340
Epoch [15/20], Accuracy: 76.39%, Loss: 0.5189
Epoch [16/20], Accuracy: 73.61%, Loss: 0.5126
Epoch [17/20], Accuracy: 77.68%, Loss: 0.4976
Epoch [18/20], Accuracy: 74.36%, Loss: 0.4892
Epoch [19/20], Accuracy: 69.88%, Loss: 0.4806
Epoch [20/20], Accuracy: 79.51%, Loss: 0.4858
Training finished.
